In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

DATA_PATH = Path('../csv_files/raw')
INGREDIENT = 'ingredient.csv'
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_row', None)

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_8632\2626821563.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# 1. (비선호음식, 원재료명)

In [ ]:
ingredient_df = pd.read_csv(DATA_PATH / INGREDIENT, encoding="utf-8")
print(f'shape: {ingredient_df.shape}')
ingredient_df.head(5)

In [5]:
def search(pname, exclude):
   if exclude:
      filtered = ingredient_df[ingredient_df['기존'].str.contains(exclude)==False]
      filtered = filtered[filtered['기존'].str.contains(pname)==True]
      print(f'{filtered.shape}')
      display(filtered)
      return

   filtered = ingredient_df[ingredient_df['기존'].str.contains(pname)==True]
   print(f'{filtered.shape}')
   display(filtered)
      

In [ ]:
# 검색 키워드: pname="게"
# 제외할 키워드: exclude="게맛살|게장"
search(pname="게", exclude=None)

# 2. (haccp의 원재료 데이터, 비선호음식)
1. (품목보고번호, 비선호음식)이 연결된 테이블 만들기
2. (품목보고번호, 원재료 string) 

### 2-1. (품목보고번호, 비선호음식)이 연결된 테이블 만들기

#### haccp

In [47]:
# 데이터 디렉토리 설정
HACCP = "HACCP.csv"
DISLIKED_TO_RAWMTR = "disliked_to_rawmtr.csv"

In [204]:
# haccp dataframe 열기
haccp_df = pd.read_csv(DATA_PATH / HACCP, encoding="utf-8")
print(f'{haccp_df.shape}')
haccp_df = haccp_df.reset_index().rename(columns={"index": "id"})
haccp_df = haccp_df.drop(columns=['item.barcode', 'item.allergy'])
haccp_df.head(5)

(15002, 5)


,id,item.prdlstNm,item.rawmtrl,item.prdlstReportNo
0,0,설화눈꽃팝김부각스낵,"찹쌀,김,참깨,옥수수기름(옥배유),양파,무,대파,천일염,마늘,새우,멸치,다시마,건표고버섯,둥굴레,감초,정제수",201904962715
1,1,설화눈꽃팝김부각스낵 아몬드맛,"찹쌀,김,참깨,옥수수기름(옥배유),아몬드,양파,무,천일염,대파,마늘,새우,멸치,다시마,건표고버섯,둥굴레,감초,정제수",201904962712
2,2,눈꽃팝김부각스낵,"찹쌀,김,참깨,옥수수기름(옥배유),양파,무,대파,천일염,마늘,새우,멸치,다시마,건표고버섯,둥굴레,감초,정제수",201904962711
3,3,참군고구마칩,군고구마,201903787252
4,4,참군고구마,군고구마,201903787251


In [205]:
# rawmtrs 다른 행으로 분리
haccp_df['item.rawmtrl'] = haccp_df['item.rawmtrl'].str.split(',')
haccp_df_exploded = haccp_df.explode('item.rawmtrl')
haccp_df_exploded.head(5)

,id,item.prdlstNm,item.rawmtrl,item.prdlstReportNo
0,0,설화눈꽃팝김부각스낵,찹쌀,201904962715
0,0,설화눈꽃팝김부각스낵,김,201904962715
0,0,설화눈꽃팝김부각스낵,참깨,201904962715
0,0,설화눈꽃팝김부각스낵,옥수수기름(옥배유),201904962715
0,0,설화눈꽃팝김부각스낵,양파,201904962715


#### disliked

In [206]:
disliked_df = pd.read_csv(DATA_PATH / DISLIKED_TO_RAWMTR, encoding="utf-8")
disliked_df = disliked_df.reset_index().rename(columns={"index": "id"})
print(f'{disliked_df.shape}')
disliked_df.head(5)

(21, 3)


,id,disliked,rawmtrs
0,0,우유/버터/치즈,"분유, 우유, 유청, 밀크, 밀키, 밀크레진, 낙우밀"
1,1,계란,"달걀, 계란, 난류, 난백, 난황, 전란, 유정란, 난각, 마요네즈, 에그, 배터믹스-EL, 배터믹스-이엘"
2,2,새우/게,"새우, 꽃게, 대게, 게맛살, 게 엑기스, 굴소스(게), 게농축액, 갑각류, 다시마게"
3,3,글루텐,"소맥분, 소맥, 밀가루, 부침가루, 빵가루, 글루텐, 밀식이섬유, 배터용분말, 치킨용배터믹스, 지미베이스, 가라아게배터, 통밀, 찐밀쌀가루, 호밀, 밀베이커스믹스, 중력분, 조미새우분말, 그릴시즈닝-22, (밀), 소밀, 밀분해액, 전분, 보리, 찰보리, 귀리"
4,4,메밀,메밀


In [207]:
# rawmtrs 다른 행으로 분리
disliked_df['rawmtrs'] = disliked_df['rawmtrs'].str.split(', ')
disliked_df_exploded = disliked_df.explode('rawmtrs')
disliked_df_exploded.head(5)

,id,disliked,rawmtrs
0,0,우유/버터/치즈,분유
0,0,우유/버터/치즈,우유
0,0,우유/버터/치즈,유청
0,0,우유/버터/치즈,밀크
0,0,우유/버터/치즈,밀키


### merge

In [214]:
merged_df = pd.merge(haccp_df_exploded, disliked_df_exploded, how='cross')
print(f'{merged_df.shape}')

(45307872, 7)


In [215]:
def merge_function(row):
    try:
        return  row['rawmtrs'] in row['item.rawmtrl']
    except:
        return False

In [216]:
result = merged_df[merged_df.apply(merge_function, axis=1)]
print(f'{result.shape}')
result.head(5)

(306506, 7)


,id_x,item.prdlstNm,item.rawmtrl,item.prdlstReportNo,id_y,disliked,rawmtrs
85,0,설화눈꽃팝김부각스낵,찹쌀,201904962715,7,소,
268,0,설화눈꽃팝김부각스낵,김,201904962715,7,소,
451,0,설화눈꽃팝김부각스낵,참깨,201904962715,7,소,
634,0,설화눈꽃팝김부각스낵,옥수수기름(옥배유),201904962715,7,소,
817,0,설화눈꽃팝김부각스낵,양파,201904962715,7,소,


In [225]:
result_nospace = result[result['rawmtrs']!='']
print(f'{result_nospace.shape}')

(58922, 7)


In [226]:
result_nospace = result_nospace.drop(columns=['id_x', 'id_y'])
result_nospace.head(10)

,item.prdlstNm,item.rawmtrl,item.prdlstReportNo,disliked,rawmtrs
1666,설화눈꽃팝김부각스낵,새우,201904962715,새우/게,새우
2378,설화눈꽃팝김부각스낵,건표고버섯,201904962715,버섯,버섯
2438,설화눈꽃팝김부각스낵,둥굴레,201904962715,오징어/어패류,굴
3778,설화눈꽃팝김부각스낵 아몬드맛,아몬드,201904962712,잣/호두/아몬드,아몬드
4777,설화눈꽃팝김부각스낵 아몬드맛,새우,201904962712,새우/게,새우
5489,설화눈꽃팝김부각스낵 아몬드맛,건표고버섯,201904962712,버섯,버섯
5549,설화눈꽃팝김부각스낵 아몬드맛,둥굴레,201904962712,오징어/어패류,굴
7705,눈꽃팝김부각스낵,새우,201904962711,새우/게,새우
8417,눈꽃팝김부각스낵,건표고버섯,201904962711,버섯,버섯
8477,눈꽃팝김부각스낵,둥굴레,201904962711,오징어/어패류,굴


In [227]:
#result_nospace.to_csv(DATA_PATH / "product_to_rawmtr.csv")

#### 마지막 정제

In [11]:
PRODUCT_TO_RAWMTR = "product_to_rawmtr.csv"
product_to_rawmtr_df = pd.read_csv(DATA_PATH / PRODUCT_TO_RAWMTR, encoding="utf-8")
product_to_rawmtr_df = product_to_rawmtr_df.drop(columns=["Unnamed: 0"])
product_to_rawmtr_df.head(3)

,item.prdlstNm,item.rawmtrl,item.prdlstReportNo,disliked,rawmtrs
0,설화눈꽃팝김부각스낵,새우,201904962715,새우/게,새우
1,설화눈꽃팝김부각스낵,건표고버섯,201904962715,버섯,버섯
2,설화눈꽃팝김부각스낵,둥굴레,201904962715,오징어/어패류,굴


In [12]:
product_to_rawmtr_df = product_to_rawmtr_df.rename(columns={"item.prdlstReportNo": "prdlstReportNo"})
product_to_rawmtr_df.head(3)

,item.prdlstNm,item.rawmtrl,prdlstReportNo,disliked,rawmtrs
0,설화눈꽃팝김부각스낵,새우,201904962715,새우/게,새우
1,설화눈꽃팝김부각스낵,건표고버섯,201904962715,버섯,버섯
2,설화눈꽃팝김부각스낵,둥굴레,201904962715,오징어/어패류,굴


In [17]:
def search(pname, exclude):
   if exclude:
      filtered = product_to_rawmtr_df[product_to_rawmtr_df['item.rawmtrl'].str.contains(exclude)==False]
      filtered = filtered[filtered['rawmtrs'].str.contains(pname)==True]
      filtered = filtered[['item.rawmtrl', 'disliked', 'rawmtrs']]
      print(f'{filtered.shape}')
      display(filtered)
      return

   filtered = product_to_rawmtr_df[product_to_rawmtr_df['rawmtrs'].str.contains(pname)==True]
   filtered = filtered[['item.rawmtrl', 'disliked', 'rawmtrs']]
   print(f'{filtered.shape}')
   display(filtered)

In [38]:
search(pname="잣", exclude=None)

(55, 3)


,item.rawmtrl,disliked,rawmtrs
23,잣,잣/호두/아몬드,잣
29,잣,잣/호두/아몬드,잣
367,잣,잣/호두/아몬드,잣
375,잣,잣/호두/아몬드,잣
3767,유기농잣(국산),잣/호두/아몬드,잣
4849,잣,잣/호두/아몬드,잣
8477,잣(국산)1%,잣/호두/아몬드,잣
9152,잣(국산)0.2%,잣/호두/아몬드,잣
23926,잣(국산),잣/호두/아몬드,잣
23936,잣(국산),잣/호두/아몬드,잣


drop 명단
1. 둥굴레 (굴)
2. 버팔로윙마리네이션 (윙)
3. 윙믹스-F (윙)
4. 아카리돈산 (돈)

In [39]:
filtered = product_to_rawmtr_df[product_to_rawmtr_df['item.rawmtrl'].str.contains("둥굴레")==False]
filtered = filtered[filtered['item.rawmtrl'].str.contains("버팔로윙마리네이션")==False]
filtered = filtered[filtered['item.rawmtrl'].str.contains("윙믹스-F")==False]
filtered = filtered[filtered['item.rawmtrl'].str.contains("아카리돈산")==False]
print(f'{filtered.shape}')
filtered.head(3)

,item.prdlstNm,item.rawmtrl,prdlstReportNo,disliked,rawmtrs
0,설화눈꽃팝김부각스낵,새우,201904962715,새우/게,새우
1,설화눈꽃팝김부각스낵,건표고버섯,201904962715,버섯,버섯
3,설화눈꽃팝김부각스낵 아몬드맛,아몬드,201904962712,잣/호두/아몬드,아몬드


In [52]:
# disliked.csv id 추가
DISLIKED = "disliked.csv"
disliked_df = pd.read_csv(DATA_PATH / DISLIKED, encoding="utf-8")
disliked_df = disliked_df.reset_index().rename(columns={"index":"id"})
disliked_df.to_csv(DATA_PATH / "disliked.csv", index=False)

In [53]:
# disliked_to_rawmtr disliked name -> index
disliked_to_rawmtr =  pd.read_csv(DATA_PATH / DISLIKED_TO_RAWMTR, encoding="utf-8")
disliked_to_rawmtr_merged = pd.merge(disliked_to_rawmtr, disliked_df, how="inner")
disliked_to_rawmtr_merged = disliked_to_rawmtr_merged.drop(columns=["disliked"])
#disliked_to_rawmtr_merged.head(4)

disliked_to_rawmtr_merged['rawmtrs'] = disliked_to_rawmtr_merged['rawmtrs'].str.split(',')
disliked_to_rawmtr_merged_exploded = disliked_to_rawmtr_merged.explode('rawmtrs')
disliked_to_rawmtr_merged_exploded = disliked_to_rawmtr_merged_exploded.rename(columns={"id":"disliked_id"})
disliked_to_rawmtr_merged_exploded = disliked_to_rawmtr_merged_exploded.reset_index(drop=True)
disliked_to_rawmtr_merged_exploded = disliked_to_rawmtr_merged_exploded.reset_index().rename(columns={"index":"id"})
disliked_to_rawmtr_merged_exploded.head(5)
#disliked_to_rawmtr_merged_exploded.to_csv(DATA_PATH / "disliked_to_mtr.csv")


,rawmtrs,id
0,분유,0
0,우유,0
0,유청,0
0,밀크,0
0,밀키,0


In [70]:
product_to_disliked = pd.merge(filtered, disliked_df, how="inner")
product_to_disliked = product_to_disliked.drop(columns=["item.prdlstNm", "item.rawmtrl", "disliked", "rawmtrs"])
product_to_disliked = product_to_disliked.rename(columns={"id":"disliked_id"})

In [75]:
product_to_disliked = product_to_disliked.reset_index().rename(columns={"index":"id"})
product_to_disliked.head(3)
# product_to_disliked.to_csv(DATA_PATH / "product_to_disliked.csv")

,id,prdlstReportNo,disliked_id
0,0,201904962715,19
1,1,201904962712,5
2,2,201904962712,19


In [76]:
product_to_disliked.to_csv(DATA_PATH / "product_to_disliked.csv")

데이터 수정

In [3]:
DATA_PATH = Path('../csv_files')
disliked_to_mtr = "disliked_to_mtr.csv"
product_to_disliked = "product_to_disliked.csv"

product_to_disliked_df = pd.read_csv(DATA_PATH / product_to_disliked, encoding="utf-8")
disliked_to_mtr_df = pd.read_csv(DATA_PATH / disliked_to_mtr, encoding="utf-8")

In [7]:
product_to_disliked_df.head(3)

,id,prdlstReportNo,disliked_id
0,0,201904962715,19
1,1,201904962712,5
2,2,201904962712,19


In [8]:
disliked_to_mtr_df.head(3)

,id,rawmtrs,disliked_id
0,0,분유,0
1,1,우유,0
2,2,유청,0


In [6]:
product_to_disliked_df = product_to_disliked_df.drop(columns=["Unnamed: 0"])
disliked_to_mtr_df = disliked_to_mtr_df.drop(columns=["Unnamed: 0"])

In [9]:
disliked_to_mtr_df['rawmtrs'] =  disliked_to_mtr_df['rawmtrs'].str.replace(" ", "")
disliked_to_mtr_df.head(3)

,id,rawmtrs,disliked_id
0,0,분유,0
1,1,우유,0
2,2,유청,0


In [11]:
product_to_disliked_df.to_csv(DATA_PATH / product_to_disliked, encoding="utf-8", index=False)
disliked_to_mtr_df.to_csv(DATA_PATH / disliked_to_mtr, encoding="utf-8", index=False)